## Zadanie 1

In [46]:
import numpy as np

def generate_chromosome(size):
    chromosome = np.zeros(size)
    for i in range(len(chromosome)):
        chromosome[i] = np.random.randint(0,2)
    
    return chromosome

def fitness(chromosome):
    return np.sum(chromosome)

def probability(p):
    return np.random.uniform() < p

def cross(population):
    fit = []
    for chromosome in population:
        res = fitness(chromosome)
        fit.append(res)
    
    #wybieramy rodziców
    p1 = np.argmax(fit)
    parent1 = population[p1]
    fit[p1] = -1
    p2 = np.argmax(fit)
    parent2 = population[p2]

    #krzyżujemy jednopunktowo
    pivot = np.random.randint(0,len(population[0]))
    child1 = np.concatenate((parent1[:pivot], parent2[pivot:]))
    child2 = np.concatenate((parent2[:pivot], parent1[pivot:]))

    #usuwamy 2 najgorsze osobniki
    fit[p1] = 11

    n = np.argmin(fit)
    fit[n] = 11
    population[n] = child1
    n = np.argmin(fit)
    population[n] = child2

    #mutacja u rodziców
    if probability(0.6):
        n = np.random.randint(0, len(population[p1]))
        if population[p1][n] == 1:
            population[p1][n] = 0
        else:
            population[p1][n] = 1
    if probability(0.6):
        n = np.random.randint(0, len(population[p2]))
        if population[p2][n] == 1:
            population[p2][n] = 0
        else:
            population[p2][n] = 1

    return population

In [3]:
number_of_genes = 10
population_size = 10
generations = 10

population = []
for i in range(population_size):
    population.append(generate_chromosome(number_of_genes))

for i in range(generations):
    population = cross(population)

fit = []
for i in range(population_size):
    fit.append(fitness(population[i]))

print(f"Najlepszy wynik po {generations} generacji: {np.max(fit)}")

Najlepszy wynik po 10 generacji: 10.0


## Zadanie 2

In [103]:
import random

def to_dec(binary_list):
    res = 0
    for i, bit in enumerate(reversed(binary_list)):
        res += bit * (2 ** i)
    return res

def fitness_quadratic(chromosome):
    a_bin = chromosome[:4]
    b_bin = chromosome[4:]
    
    a = to_dec(a_bin)
    b = to_dec(b_bin)

    res = 33 - 2*a*a - b

    return 500 - abs(res)

def cross_roulette(population):
    size = len(population)
    new_population = []
    fit = [fitness_quadratic(chromosome) for chromosome in population]
    fit_sum = np.sum(fit)

    #nowa populacja
    population_probability = [f/fit_sum for f in fit]
    new_population = random.choices(population, weights=population_probability, k=size)
    #print(new_population)

    #krzyżowanie
    childs = new_population
    for i in range(size//2):
        p1 = np.random.randint(0,size)
        p2 = np.random.randint(0,size)
        parent1 = new_population[p1]
        parent2 = new_population[p2]

        pivot = np.random.randint(0, size)
        child = np.concatenate((parent1[:pivot], parent2[pivot:]))
        
        childs.append(child)

    #mutacje
    for i in range(size):
        if probability(0.1):
            n = np.random.randint(0, len(child))
            if childs[i][n] == 1:
                childs[i][n] = 0
            else:
                childs[i][n] = 1
        
    return childs

In [117]:
number_of_genes = 8
population_size = 10
generations = 20

population = []
for i in range(population_size):
    population.append(generate_chromosome(number_of_genes))

for i in range(generations):
    population = cross_roulette(population)
    
fit = [fitness_quadratic(f) for f in population]

print(f"Najlepszy wynik po {generations} generacji: {np.max(fit)} (500 punktów to max)\n {population[np.argmax(fit)]}")

Najlepszy wynik po 20 generacji: 500.0 (500 punktów to max)
 [0. 1. 0. 0. 0. 0. 0. 1.]


## Zadanie 3

In [154]:
def fitness_backpack(chromosome, weights, values):
    res = 0
    weight = 0
    for i, c in enumerate(chromosome):
        if c == 1:
            weight += weights[i]
            res += values[i]
        
        if weight > 35:
            res = 0
            break

    return res

def cross_backpack(population, weights, values):
    size = len(population)
    fit = [fitness_backpack(chromosome, weights, values) for chromosome in population]
    new_population = []
    #kopiujemy 25% najlepszych osobników
    for i in range(size//4):
        n = np.argmax(population)
        chromosome = population[n]
        new_population.append(chromosome)

        population.pop(n)
        fit.pop(n)
    
    #ruletka
    fit = [fitness_backpack(chromosome, weights, values) for chromosome in population]
    size -= len(new_population)
    fit_sum = np.sum(fit)
    population_probability = [f/fit_sum for f in fit]
    tmp_population = random.choices(population, weights=population_probability, k=size)
    
    
    #krzyżowanie
    for i in range(size//2):
        p1, p2 = np.random.choice(len(tmp_population), size=2, replace=False)
        parent1 = tmp_population[p1]
        parent2 = tmp_population[p2]

        pivot = np.random.randint(0, size)
        child = np.concatenate((parent1[:pivot], parent2[pivot:]))
        
        #zastępujemy rodzica
        p = p1
        if pivot%2 == 0:
            p = p2
        tmp_population.pop(p)
        
        #mutacja dziecka
        for i in child:
            if probability(0.05):
                if i == 1:
                    i = 0
                else:
                    i = 1

        new_population.append(child)
    new_population.extend(tmp_population)
    
    return new_population

In [163]:
number_of_genes = 10
population_size = 8
generations = 10

weight = [3,13,10,9,7,1,8,8,2,9]
value = [266,442,671,526,388,245,210,145,126,322]

population = []
for i in range(population_size):
    population.append(generate_chromosome(number_of_genes))

for i in range(generations):
    population = cross_backpack(population, weight, value)

fit = []
for i in range(population_size):
    fit.append(fitness_backpack(population[i], weight, value))

print(f"Najlepszy wynik po {generations} generacji: {np.max(fit)}\n{population[np.argmax(fit)]}")

Najlepszy wynik po 10 generacji: 1453
[1. 0. 1. 0. 0. 1. 0. 1. 1. 0.]


Najlepszy wynik po 10 generacji: 2156
[1. 0. 1. 1. 0. 1. 0. 0. 1. 1.]

Najlepszy wynik po 10 generacji: 2222
[1. 0. 1. 1. 1. 1. 0. 0. 1. 0.]

## Zadanie 4

In [ ]:
def generate_xy_chromosome(x, y):
    chromosome = []
    visited = []
    size = len(x)
    while len(visited) != size:
        index = np.random.randint(0,size)
        if index in visited:
            continue

        gene = (x[index], y[index])
        chromosome.append(gene)
        visited.append(index)
    
    return chromosome

def distance(chromosome):
    dist = 0
    for i in range(len(chromosome)-1):
        x1, y1 = chromosome[i]
        x2, y2 = chromosome[i+1]

        dist += ((x1-x2)**2 + (y1-y2)**2)**0.5

    #powrót z ostatniego punktu do pierwszego
    x_first, y_first = chromosome[0]
    x_last, y_last = chromosome[-1]
    dist += ((x_first-x_last)**2 + (y_first-y_last)**2)**0.5

    return 10000 - dist

def the_same_place(ch1, ch2):
    x1, y1 = ch1
    x2, y2 = ch2

    return x1==x2 and y1==y2

def cross_tsp(population):
    size = len(population)
    number_of_genes = len(population[0])
    fit = [distance(chromosome) for chromosome in population]
    new_population = []
    #kopiujemy 20% najlepszych osobników
    for i in range(size//5):
        n = np.argmax(fit)
        chromosome = population[n]
        new_population.append(chromosome)

        population.pop(n)
        fit.pop(n)
    
    #ruletka
    fit = [distance(chromosome) for chromosome in population]
    size -= len(new_population)
    fit_sum = np.sum(fit)
    population_probability = [f/fit_sum for f in fit]
    tmp_population = random.choices(population, weights=population_probability, k=size)
    
    
    #krzyżowanie (operator krzyżowania uporządkowanego)
    for i in range(size//2):
        p1, p2 = np.random.choice(len(tmp_population), size=2, replace=False)
        parent1 = tmp_population[p1]
        parent2 = tmp_population[p2]

        try:
            pivot1 = np.random.randint(4, number_of_genes//2 - 3)
        except:
            print(number_of_genes)
            break
        pivot2 = np.random.randint(number_of_genes//2 + 3, number_of_genes-4)
        middle_part = []
        tmp = list(parent1[:pivot1])
        for g in parent1[pivot2:]:
            tmp.append(g)
        for gene in parent2:
            for g in tmp:
                if the_same_place(gene, g):
                    continue

            middle_part.append(gene)
            tmp.append(gene)
        
        try:
            child = np.concatenate((parent1[:pivot1], middle_part, parent1[pivot2:]))
        except:
            print(f"p1={pivot1},p2={pivot2}")
            print("p1\n",parent1[:pivot1])
            print("middle_part\n",middle_part)
            print("p1\n",parent1[pivot2:])
        
        #zastępujemy rodzica
        p = p1
        if pivot1%2 == 0:
            p = p2
        tmp_population.pop(p)
        
        #mutacja dziecka
        for index, i in enumerate(child):
            if probability(0.01):
                if all(i == child[-1]):
                    n = child[index]
                    child[index] = child[index-1]
                    child[index-1] = n
                else:
                    n = child[index]
                    child[index] = child[index+1]
                    child[index+1] = n

        new_population.append(child)
    new_population.extend(tmp_population)
    
    return new_population

In [225]:
number_of_genes = 25
population_size = 100
generations = 1000

x = [119,37, 197, 85,12,100,81,121,85,80,91,106,123,40,78,190,187,37,17,67,78,87,184,111,66]
y = [38,48,55,165,50,53,142,137,145,197,176,55,57,81,125,46,40,107,11,56,133,23,187,12,178]

#chromosom: (x,y)
population = []
for i in range(population_size):
    population.append(generate_xy_chromosome(x,y))

for i in range(generations):
    population = cross_tsp(population)

fit = []
for i in range(population_size):
    fit.append(distance(population[i]))

print(f"Najlepszy wynik po {generations} generacji: {10000 - round(np.max(fit),2)}")


Najlepszy wynik po 1000 generacji: 7910.33
